This notebook shows a simple demo of how to use Aug-imodels (`AugLinearClassifier` and `AugTreeClassifier`). These models use LLMs to augment trnsparent models (e.g. GAMs and decision trees) to improve their performance.

Both follow a simple sklearn-style interface, but may be slow to fit (because of the LLM augmentation). Both are extremely fast at test time, as they no longer use an LLM.

In [1]:
%load_ext autoreload
%autoreload 2
from imodelsx import AugLinearClassifier, AugTreeClassifier
import datasets
import numpy as np

[2024-02-12 11:16:31,412] [INFO] [real_accelerator.py:158:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/thinc/compat.py:36: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  hasattr(torch, "has_mps")
/home/chansingh/imodelsx/.venv/lib/python3.11/site-packages/thinc/compat.py:37: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  and torch.has_mps  # type: ignore[attr-defined]


### Load some data
Here, we load some training/validation data from the rotten-tomatoes movie dataset. To make things fast, we restrict our training and testing datasets to only 300 examples.

In [2]:
dset = datasets.load_dataset('rotten_tomatoes')['train']
dset = dset.select(np.random.choice(len(dset), size=300, replace=False))

dset_val = datasets.load_dataset('rotten_tomatoes')['validation']
dset_val = dset_val.select(np.random.choice(
    len(dset_val), size=300, replace=False))

# Aug-Linear

### Fit AugLinearClassifier
Fitting AugLinear is a simple function call! AugLinear takes a few hyperparameters.

In [4]:
m = AugLinearClassifier(
    checkpoint='textattack/distilbert-base-uncased-rotten-tomatoes',
    ngrams=2,
    all_ngrams=True,  # also use lower-order ngrams
)
m.fit(dset['text'], dset['label'])

initializing model...
calculating embeddings...


100%|██████████| 300/300 [01:02<00:00,  4.81it/s]


set up linear model...
caching linear coefs...


100%|██████████| 15/15 [00:38<00:00,  2.57s/it]

	After caching, len(coefs_dict_)=7208, up from 0


AugLinearClassifier(all_ngrams=True,
                    checkpoint='textattack/distilbert-base-uncased-rotten-tomatoes',
                    tokenizer_ngrams=<spacy.tokenizer.Tokenizer object at 0x7f210c14d750>)

### Interpretation

We now have a linear model of ngrams. The `fit` function above has precomputed the linear coefficients for ngrams it saw during training and saved them to `m.coefs_dict_` Let's take a look at some of them.

In [ ]:
print('Total ngram coefficients: ', len(m.coefs_dict_))
print('Most positive ngrams')
for k, v in sorted(m.coefs_dict_.items(), key=lambda item: item[1], reverse=True)[:8]:
    print('\t', k, round(v, 2))
print('Most negative ngrams')
for k, v in sorted(m.coefs_dict_.items(), key=lambda item: item[1])[:8]:
    print('\t', k, round(v, 2))

### Predictions
Now, let's take a look at how we make predictions. This is very fast, as it just uses the precomputed dictionary `m.coefs_dict_`

In [5]:
preds = m.predict(dset['text'])
print('acc_train', np.mean(preds == dset['label']))
preds_proba = m.predict_proba(dset['text'])

acc_train 0.8066666666666666


In [6]:
preds = m.predict(dset_val['text'])
print('acc_val', np.mean(preds == dset_val['label']))

acc_val 0.6566666666666666


/home/chansingh/imodelsx/imodelsx/auglinear/auglinear.py:375: UserWarning: Saw an unseen ungram 6651 times. For better performance, call cache_linear_coefs on the test dataset before calling predict.
  f"Saw an unseen ungram {n_unseen_ngrams} times. \


Note: we may want to infer the coefficients for ngrams we didn't see during training. To do this, we call the `cache_linear_coefs` function on the inputs for the test set. This adds the values for the unseen coefficients to the dictionary `m.coefs_dict_`. Then we can call `predict` as before.

In [7]:
m.cache_linear_coefs(dset_val['text'])
preds = m.predict(dset_val['text'])
print('acc_val', np.mean(preds == dset_val['label']))

100%|██████████| 13/13 [00:32<00:00,  2.50s/it]

	After caching, len(coefs_dict_)=13412, up from 7208
acc_val 0.7366666666666667


# Aug-Tree

In [ ]:
import imodelsx.augtree.data

# set  your openai key
import openai
openai.api_key = open('/home/chansingh/.OPENAI_KEY', 'r').read().strip()

# pepare data
X_text = list(dset['text'])
# optionally, convert data to ngrams
X, _, feature_names = imodelsx.augtree.data.convert_text_data_to_counts_array(
    X_text, [], ngrams=2)

In [ ]:
m = AugTreeClassifier(
    max_depth=2,  # depth of the tree
    max_features=1,
    # this tells the classifier to actually use the llm (defaults to text-davinci-003)
    refinement_strategy='llm',
    verbose=True,
    # folder to store cached ngram expansions
    cache_expansions_dir='/home/chansigh/aug-models/augtree/results/gpt3_cache',
)
m.fit(X=X, y=dset['label'], feature_names=feature_names, X_text=X_text)

In [ ]:
print(m)